In [10]:
# 서버와의 요청 및 응답을 위한 라이브러리
import requests as req 
# 컴퓨터가 이해하기 쉬운 객체로 변환해주는 라이브러리
from bs4 import BeautifulSoup as bs
# 데이터프레임으로 형식으로 만들기 위한 라이브러리
import pandas as pd
# 진행 상황을 표시하기 위한 라이브러리
from tqdm import tqdm_notebook as tq
# webdriver -> 웹 브라우저를 제어하기 위한 라이브러리
from selenium import webdriver as wb
# 웹을 통해서 값을 '입력'하기 위한 라이브러리 -> 컴퓨터용 키보드
from selenium.webdriver.common.keys import Keys
# 태그의 위치를 찾기 위한 라이브러리
from selenium.webdriver.common.by import By
# 코딩 중간에 쉬는시간을 부여
import time
# 파일이나 폴더를 생성해주는 라이브러리(파일 시스템 라이브러리)
import os
# 이미지 경로를 파일로 변환해주는 라이브러리
from urllib.request import urlretrieve
#정규 표현식을 사용
import re

In [42]:
# Selenium 웹 드라이버 초기화
driver = wb.Chrome()

# 크롤링할 웹 사이트의 베이스 URL
base_url ='https://www.masileng.com/cocktail/'

# 각각의 정보를 저장할 리스트들
name_list = []
alc_list = []
explanation_list = []
recipe_list = []
recipe_num = []
# 칵테일 재료와 양을 저장할 리스트들
ingredient_list = []
volume_list = []
ingre_num=[]
ingre_cock_list = []
img_list = []
img_num = []
img_cock_num=[]
for i in tq(range(1,73)) :
    # 페이지 URL 조합
    url = f'{base_url}{i}'
    # 웹 페이지 접속
    driver.get(url)
    # BeautifulSoup으로 페이지 소스 파싱
    soup = bs(driver.page_source,'lxml')
    # 칵테일 이름 추출 및 리스트에 추가
    name = soup.select_one('#__next > div.bubbling-limit > main > div > div.cocktail-detail__Container-sc-sw6wpa-0.iSATes.style_custom-detail-page__nvS_Y > div > div.cocktail-detail__DescriptionContainer-sc-sw6wpa-7.hJFmQn.style_detail-txt__P9_7f > div.cocktail-detail__TitleContainer-sc-sw6wpa-8.jRYmtR.style_title-wrap__aX5Lc > div.text__Text-sc-1y4jww2-0.rdOSR.detail-item__Name-sc-c6hn19-0.dMaghw')
    name_list.append(name.text)
    # 알코올 도수 추출 및 리스트에 추가
    alc = soup.select_one('#__next > div.bubbling-limit > main > div > div.cocktail-detail__Container-sc-sw6wpa-0.iSATes.style_custom-detail-page__nvS_Y > div > div.cocktail-detail__DescriptionContainer-sc-sw6wpa-7.hJFmQn.style_detail-txt__P9_7f > div.cocktail-detail__Flex-sc-sw6wpa-9.fnhXaf.style_sub-wrap__OSZhB > div')
    alc_list.append(alc.text)
    # 칵테일 설명 추출 및 리스트에 추가
    explanation = soup.select_one('#__next > div.bubbling-limit > main > div > div.cocktail-detail__Container-sc-sw6wpa-0.iSATes.style_custom-detail-page__nvS_Y > div > div.cocktail-detail__DescriptionContainer-sc-sw6wpa-7.hJFmQn.style_detail-txt__P9_7f > div.text__Text-sc-1y4jww2-0.rdOSR.detail-item__Description-sc-c6hn19-2.cRBObK')
    explanation_list.append(explanation.text)
    # 칵테일 레시피 추출 및 리스트에 추가
    recipe = soup.select_one('#__next > div.bubbling-limit > main > div > div:nth-child(4) > div > p')
    recipe_list.append(recipe.text)
    # 해당 페이지의 칵테일 번호 저장
    recipe_num.append('R' + str(i))
    # 칵테일 재료 및 양의 해당하는 칵테일 번호 저장
    current_cocknum = f'R{i}'
    # 2초 대기
    time.sleep(2)
    
    # 칵테일 재료 및 양 추출
    ingredient = soup.find_all("div", class_="text__Text-sc-1y4jww2-0 rdOSR ingredient-item__Name-sc-11leu5j-6 gDYVNy")
    volume = soup.find_all("div", class_="text__Text-sc-1y4jww2-0 rdOSR ingredient-item__Amount-sc-11leu5j-8 bKKCmW")
    # 칵테일 재료와 양을 추출하여 리스트에 추가하고 현재 레시피번호 저장
    for i in range(len(ingredient)) :
        ingre_cock_list.append(current_cocknum)
        ingredient_list.append(ingredient[i].text)
        volume_list.append(volume[i].text)
    ingre_dic ={'레시피 번호' : ingre_cock_list, '재료명' : ingredient_list,'용량' : volume_list}
    # 칵테일 이미지의 src를 추출하여 리스트에 추가하고 현재 레시피번호 저장
    cock_img =soup.find_all("div", class_='image-previewer__Item-sc-mdbr0z-1 dzFMXb')
    for i in cock_img:
        img_tag=i.find("img")
        if img_tag:
            src=img_tag.get("src")
            img_list.append(src)
            img_cock_num.append(current_cocknum)
    img_dic = {'레시피 번호':img_cock_num,'이미지경로' : img_list}
# 이미지리스트의 길이만큼 이미지번호 리스트 생성
for i in range(len(img_list)):
    img_num.append('IMG' + str(i+1))
# 재료리스트의 길이만큼 재료번호 리스트 생성
for i in range(len(ingredient_list)) :
    ingre_num.append('IN' + str(i+1))    
# 도수를 변환할 빈 리스트
result= []
# 각 항목에서 숫자를 추출하여 리스트에 저장
for item in alc_list:
    match = re.search(r'\d+', item)  # 정규 표현식을 사용하여 문자열에서 숫자를 찾음
    if match:
        result.append(int(match.group()))  # 숫자를 정수로 변환하여 결과 리스트에 추가
    else:
        result.append(0)  # 숫자가 없는 경우 0을 결과 리스트에 추가

C:\Users\smhrd\AppData\Local\Temp\ipykernel_9604\3464377573.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tq(range(1,73)) :


  0%|          | 0/72 [00:00<?, ?it/s]

In [43]:
ingre_data = pd.DataFrame(ingre_dic, index = ingre_num)
ingre_data

,레시피 번호,재료명,용량
IN1,R1,사과,1slice
IN2,R1,얼음,100gram
IN3,R1,사과 리큐르,15ml
IN4,R1,쿠앵트로,15ml
IN5,R1,보드카,40ml
...,...,...,...
IN360,R72,얼음,100gram
IN361,R72,마라스키노 체리,1개
IN362,R72,파인애플,1wedge
IN363,R72,파인애플 주스,120ml


In [44]:
img_data = pd.DataFrame(img_dic, index = img_num)
img_data

,레시피 번호,이미지경로
IMG1,R1,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG2,R2,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG3,R2,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG4,R3,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG5,R3,https://cocktail-bucket.s3.ap-northeast-2.amaz...
...,...,...
IMG133,R70,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG134,R70,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG135,R71,https://cocktail-bucket.s3.ap-northeast-2.amaz...
IMG136,R71,https://cocktail-bucket.s3.ap-northeast-2.amaz...


In [46]:
dic = {'레시피 이름' : name_list, '도수' : result, '설명' : explanation_list, '제조방법' : recipe_list}
recipe_data = pd.DataFrame(dic, index = recipe_num)
recipe_data

,레시피 이름,도수,설명,제조방법
R1,애플 마티니,35,애플티니(Appletini)라고도 불리며 보드카 대신 화이트럼을 사용하기도 합니다.,"1. 믹싱 글라스에 얼음을 넣는다. \n2. 보드카 40ml, 사과 리큐르 15ml..."
R2,B-52,26,미국의 폭격기 B-52에서 이름을 따온 칵테일로 뚜렷한 3개의 층이 특징입니다. 얇...,1. 슈터 글라스에 칼루아 20ml를 붓는다. \n2. 칼루아 위에 베일리스를 아주...
R3,바카디 칵테일,33,주로 바카디 슈페리어로 만들고 식전주로 즐깁니다.,"1. 셰이커에 각얼음을 채운다. \n2. 화이트 럼 45ml, 라임 주스 20ml,..."
R4,블랙 러시안,33,러시아의 전형적인 술인 보드카를 사용하고 커피 리큐르의 검은색을 사용하여 그 이름을...,1. 락글라스에 얼음을 채운다. \n2. 얼음 위에 보드카 50ml와 커피 리큐르 ...
R5,블러디 메리,13,피의 메리라는 이름은 영국의 메리 1세 여왕의 이름에서 비롯되었다고도 합니다.,"1. 하이볼 글라스에 얼음을 넣는다. \n2. 우스터소스 2-3대쉬, 타바스코 1대..."
...,...,...,...,...
R68,고진감래주,4,고생이 다하면 즐거움이 찾아온다는 한자성어 고진감래의 의미를 가진 폭탄주. 마시면 ...,1. 소주잔 1에 콜라를 반만 따라서 맥주잔 안에 넣는다. \n2. 소주잔 2에 소...
R69,한라토닉,6,소주로 쉽게 만들 수 있는 하이볼. 제주도의 한라산 소주를 사용하면 더 맛이 좋습니...,1. 하이볼잔에 각얼음을 넣는다. \n2. 얼음위에 소주와 토닉워터를 1:2의 비율...
R70,로이 로저스,0,미국의 배우이자 가수인 로이 로저스의 이름을 딴 상쾌한 무알콜 칵테일입니다.,1. 하이볼 글라스에 얼음을 넣는다. \n2. 그레나딘 시럽 10ml를 붓고 잔에 ...
R71,셜리 템플,0,미국의 아역배우 셜리 템플을 위해 만들어진 심플하고 대중적인 무알콜 칵테일입니다.,1. 콜린스 잔에 얼음을 채운다. \n2. 진저 에일 60ml와 레몬에이드 60ml...


In [49]:
ingre_data.to_csv('재료테이블.csv', encoding = 'UTF-8')
img_data.to_csv('이미지테이블.csv',encoding = 'UTF-8')
recipe_data.to_csv('레시피테이블.csv',encoding = 'UTF-8')